## Knapsack problem

### Create knapsack problem

In [1]:
from QHyper.problems.knapsack import KnapsackProblem

knapsack = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
print(knapsack.knapsack.items)

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [2]:
print(f"Variables: {knapsack.variables}")
print(f"Objective function: {knapsack.objective_function}")
print("Constraints:")
for constraint in knapsack.constraints:
    print(f"    {constraint}")

Variables: (x0, x1, x2, x3, x4)
Objective function: -2*x0 - 2*x1 - x2
Constraints:
    -x3 - x4 + 1
    -x0 - x1 - x2 + x3 + 2*x4


### Use VQA to solve knapsack problem

In [3]:
import numpy as np

from QHyper.solvers.vqa.base import VQA
from QHyper.solvers.vqa import QAOA, HQAOA
from QHyper.optimizers.random import Random
from QHyper.optimizers.basinhopping import Basinhopping

In [4]:
tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(knapsack, config=tester_config)

In [5]:
solver_config = {
    # 'optimizer': {
    #     'type': 'scipy',
    #     'maxfun': 200,
    # },
    'optimizer': {
        'type': 'basinhopping',
        'maxfun': 200,
        'niter': 10,
        'bounds': [(1, 10)]*3 + [(0, 2*np.pi)]*10
    },
    'pqc': {
        'type': 'hqaoa',
        'layers': 5,
    },
}

params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}

vqa = VQA(knapsack, config=solver_config)
best_params = vqa.solve(params_cofing)

In [7]:
best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer: {best_params['angles']},\nand params used for hyper optimizer: {best_params['hyper_args']}")
# We haven't use hyper optimizer and qaoa only optimize angles, hence hyperparameters are the same as provided

Best results: -1.7529002241121714
Params used for optimizer: [0.         6.24584435 6.19718055 2.04922327 6.27942849 1.25562253
 2.54497514 0.02608649 1.2461896  1.57247997],
and params used for hyper optimizer: [1.         2.84091394 8.52015871]


In [8]:
# You can provide VQA with objects instead of cofinguration dict
# Here we create VQA instance, without optimizer, which will results geting just a quantum circuit (no optimization inside)
vqa = VQA(knapsack, QAOA(layers=5), None)
best_result, _, _ = vqa.solve(best_params)
print(f"Results are the same as above: {best_result}")

ValueError: not enough values to unpack (expected 3, got 2)

#### Using hyper optimizers

In [7]:
from QHyper.optimizers.random import Random

solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    },
    'eval_func': {
        'type': 'expval'
    }
}
params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}
vqa = VQA(knapsack, config=solver_config)

random = Random(processes=5, number_of_samples=100, bounds=[[1, 10], [1, 10], [1, 10]])
best_result_h, optimizer_params_h, hyper_params_h = vqa.solve(params_cofing, random)

100%|██████████| 100/100 [26:24<00:00, 15.84s/it]


In [8]:
print(f"Best results from hyper optimizer: {best_result_h}")
print(f"Params used for optimizer: {optimizer_params_h},\nand params used for hyper optimizer: {hyper_params_h}")

Best results from hyper optimizer: -41.0438258707136
Params used for optimizer: [[0.0238681  5.89321814 6.17761962 0.         2.6826312 ]
 [0.         0.55771513 0.13841942 6.11464352 6.0394204 ]],
and params used for hyper optimizer: [9.41620504 4.94209924 2.11245363]


In [9]:
params_cofing = {
    'angles': optimizer_params_h,
    'hyper_args': hyper_params_h,
}
# You can provide VQA with objects instead of cofinguration dict
# Here we create VQA instance, without optimizer, which will results geting just a quantum circuit (no optimization inside)
vqa = VQA(knapsack, QAOA(layers=5), None, WFEval())
best_result, _, _ = vqa.solve(params_cofing)
print(f"Results are the same as above: {best_result}")

Results are the same as above: -0.06004883742587043
